# Airline Delay Economics Analysis
## Which Airlines Deliver the Best Value for Money?

**Goal:** Translate flight delay data into an economic efficiency metric to determine which airlines and airports offer the best "value for money" in on-time performance.

**Dataset:** Bureau of Transportation Statistics — Airline On-Time Performance Data

**Analysis Pipeline:**
1. Data Loading & Cleaning
2. Feature Computation (delay costs, efficiency metrics)
3. Aggregation (airline-level and airport-level summaries)
4. Regression Analysis (distance vs delay relationship)
5. Export for Tableau visualization

## 1. Setup & Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports complete")

✓ Imports complete


## 2. Data Loading

**Note:** Place your BTS CSV file in the `../data/` directory.

Expected columns:
- `Carrier` (airline code)
- `Origin`, `Dest` (airport codes)
- `ArrDelay`, `DepDelay` (in minutes)
- `Distance` (in miles)
- `Month`, `Year`

In [2]:
# Define columns to load (memory efficient)
columns_to_load = [
    'Year', 'Month', 'DayofMonth',
    'Carrier', 'Origin', 'Dest',
    'ArrDelay', 'DepDelay', 'Distance',
    'Cancelled', 'Diverted'
]

# Load data
# NOTE: Update the filename to match your downloaded BTS data
data_file = '../data/airline_ontime.csv'  # Update this path

try:
    df = pd.read_csv(data_file, usecols=columns_to_load, low_memory=False)
    print(f"✓ Data loaded: {len(df):,} rows")
    print(f"✓ Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
except FileNotFoundError:
    print("❌ Data file not found. Please download BTS data and place it in ../data/")
    print("   Download from: https://catalog.data.gov/dataset/airline-on-time-performance-data")
    df = None

ValueError: Usecols do not match columns, columns expected but not found: ['ArrDelay', 'Carrier', 'Distance', 'Cancelled', 'Diverted', 'Origin', 'Month', 'Dest', 'DepDelay', 'DayofMonth', 'Year']

In [ ]:
# Display first few rows and basic info
if df is not None:
    display(df.head())
    print("\nDataset Info:")
    df.info()

## 3. Data Cleaning

In [ ]:
if df is not None:
    print(f"Starting rows: {len(df):,}\n")
    
    # Step 1: Remove cancelled and diverted flights
    df_clean = df[(df['Cancelled'] == 0) & (df['Diverted'] == 0)].copy()
    print(f"After removing cancelled/diverted: {len(df_clean):,} rows")
    
    # Step 2: Remove rows with missing delay values
    df_clean = df_clean.dropna(subset=['ArrDelay', 'DepDelay', 'Distance'])
    print(f"After removing missing values: {len(df_clean):,} rows")
    
    # Step 3: Filter out extreme outliers (delays > 500 minutes or < -60 minutes)
    df_clean = df_clean[
        (df_clean['ArrDelay'] >= -60) & 
        (df_clean['ArrDelay'] <= 500) &
        (df_clean['Distance'] > 0)
    ]
    print(f"After removing outliers: {len(df_clean):,} rows")
    
    # Step 4: Filter to top carriers and airports for MVP
    top_carriers = df_clean['Carrier'].value_counts().head(5).index
    top_airports = df_clean['Origin'].value_counts().head(10).index
    
    df_mvp = df_clean[
        df_clean['Carrier'].isin(top_carriers) &
        df_clean['Origin'].isin(top_airports)
    ].copy()
    
    print(f"\nMVP subset (top 5 carriers, top 10 airports): {len(df_mvp):,} rows")
    print(f"\nTop 5 Carriers: {list(top_carriers)}")
    print(f"Top 10 Airports: {list(top_airports)}")
    
    # Clean up
    df = df_mvp
    del df_clean, df_mvp
    
    print("\n✓ Data cleaning complete")

## 4. Feature Computation

Create economic efficiency metrics:
- `delay_cost`: Cost in USD based on FAA estimate ($74/minute)
- `cost_per_mile`: Normalized efficiency metric
- `is_delayed`: Binary flag for delays > 15 minutes

In [ ]:
if df is not None:
    # FAA estimated cost per minute of delay
    COST_PER_MINUTE = 74
    
    # Compute delay cost (only for positive delays)
    df['delay_cost'] = df['ArrDelay'].apply(lambda x: max(0, x) * COST_PER_MINUTE)
    
    # Compute cost per mile (normalized efficiency metric)
    df['cost_per_mile'] = df['delay_cost'] / df['Distance']
    
    # Binary flag for significant delays (> 15 minutes)
    df['is_delayed'] = (df['ArrDelay'] > 15).astype(int)
    
    # Display sample of new features
    print("Sample of computed features:\n")
    display(df[['Carrier', 'Origin', 'ArrDelay', 'Distance', 'delay_cost', 'cost_per_mile', 'is_delayed']].head(10))
    
    print("\n✓ Feature computation complete")

## 5. Airline-Level Aggregation

In [ ]:
if df is not None:
    airline_summary = df.groupby('Carrier').agg({
        'ArrDelay': ['mean', 'median', 'std'],
        'delay_cost': ['mean', 'sum'],
        'cost_per_mile': 'mean',
        'is_delayed': ['sum', 'mean'],  # count and rate
        'Distance': 'mean',
        'Carrier': 'count'  # number of flights
    }).reset_index()
    
    # Flatten column names
    airline_summary.columns = [
        'Carrier', 
        'avg_delay_min', 'median_delay_min', 'std_delay_min',
        'avg_delay_cost', 'total_delay_cost',
        'avg_cost_per_mile',
        'num_delayed_flights', 'delay_rate',
        'avg_distance',
        'num_flights'
    ]
    
    # Sort by cost per mile (efficiency)
    airline_summary = airline_summary.sort_values('avg_cost_per_mile')
    
    print("Airline Performance Summary:\n")
    display(airline_summary)
    
    # Export for Tableau
    airline_summary.to_csv('../outputs/airline_summary.csv', index=False)
    print("\n✓ Airline summary exported to outputs/airline_summary.csv")

## 6. Airport-Level Aggregation

In [ ]:
if df is not None:
    airport_summary = df.groupby('Origin').agg({
        'ArrDelay': ['mean', 'median'],
        'delay_cost': ['mean', 'sum'],
        'cost_per_mile': 'mean',
        'is_delayed': 'mean',
        'Origin': 'count'
    }).reset_index()
    
    # Flatten column names
    airport_summary.columns = [
        'Airport',
        'avg_delay_min', 'median_delay_min',
        'avg_delay_cost', 'total_delay_cost',
        'avg_cost_per_mile',
        'delay_rate',
        'num_flights'
    ]
    
    # Sort by cost per mile
    airport_summary = airport_summary.sort_values('avg_cost_per_mile')
    
    print("Airport Performance Summary:\n")
    display(airport_summary)
    
    # Export for Tableau
    airport_summary.to_csv('../outputs/airport_summary.csv', index=False)
    print("\n✓ Airport summary exported to outputs/airport_summary.csv")

## 7. Simple Linear Regression: Distance vs Delay

**Question:** Does flight distance predict arrival delay?

This provides analytical rigor by testing whether longer flights experience systematically different delays.

In [ ]:
if df is not None:
    # Prepare data for regression
    X = df[['Distance']].values
    y = df['ArrDelay'].values
    
    # Fit linear regression model
    model = LinearRegression()
    model.fit(X, y)
    
    # Get predictions and metrics
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    
    # Print results
    print("Linear Regression Results: ArrDelay ~ Distance\n")
    print(f"Intercept: {model.intercept_:.2f} minutes")
    print(f"Slope: {model.coef_[0]:.4f} minutes per mile")
    print(f"  → Extra delay per 100 miles: {model.coef_[0] * 100:.2f} minutes")
    print(f"R²: {r2:.4f}")
    print(f"  → Distance explains {r2*100:.2f}% of delay variability")
    
    # Interpretation
    print("\n📊 Interpretation:")
    if model.coef_[0] > 0:
        print(f"  • Longer flights tend to have slightly higher delays")
        print(f"  • For every 100 miles, delays increase by ~{model.coef_[0] * 100:.1f} minutes")
    else:
        print(f"  • Longer flights tend to have slightly lower delays (possibly due to time cushion)")
    
    if r2 < 0.05:
        print(f"  • Distance is a weak predictor (R²={r2:.3f})")
        print(f"  • Other factors (weather, congestion, carrier) likely more important")
    
    print("\n✓ Regression analysis complete")

## 8. Visualization: Regression Plot

In [ ]:
if df is not None:
    # Create scatter plot with regression line
    plt.figure(figsize=(12, 6))
    
    # Sample for visualization (too many points otherwise)
    sample_df = df.sample(min(5000, len(df)), random_state=42)
    
    plt.scatter(sample_df['Distance'], sample_df['ArrDelay'], 
                alpha=0.3, s=10, label='Actual delays')
    
    # Plot regression line
    X_range = np.array([[df['Distance'].min()], [df['Distance'].max()]])
    y_range = model.predict(X_range)
    plt.plot(X_range, y_range, 'r-', linewidth=2, 
             label=f'Regression line (R²={r2:.3f})')
    
    plt.xlabel('Distance (miles)', fontsize=12)
    plt.ylabel('Arrival Delay (minutes)', fontsize=12)
    plt.title('Flight Distance vs Arrival Delay', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../outputs/regression_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Regression plot saved to outputs/regression_plot.png")

## 9. Summary Statistics & Key Insights

In [ ]:
if df is not None:
    print("="*60)
    print("KEY INSIGHTS SUMMARY")
    print("="*60)
    
    # Overall statistics
    total_flights = len(df)
    total_delay_cost = df['delay_cost'].sum()
    avg_delay = df['ArrDelay'].mean()
    delay_rate = df['is_delayed'].mean()
    
    print(f"\n📈 OVERALL STATISTICS:")
    print(f"  • Total flights analyzed: {total_flights:,}")
    print(f"  • Average arrival delay: {avg_delay:.1f} minutes")
    print(f"  • Delay rate (>15 min): {delay_rate*100:.1f}%")
    print(f"  • Total delay cost: ${total_delay_cost:,.0f}")
    print(f"  • Average cost per flight: ${total_delay_cost/total_flights:.2f}")
    
    # Best and worst carriers
    best_carrier = airline_summary.iloc[0]
    worst_carrier = airline_summary.iloc[-1]
    
    print(f"\n🏆 BEST CARRIER (by cost per mile):")
    print(f"  • {best_carrier['Carrier']}")
    print(f"  • Avg delay: {best_carrier['avg_delay_min']:.1f} minutes")
    print(f"  • Cost per mile: ${best_carrier['avg_cost_per_mile']:.2f}")
    print(f"  • Delay rate: {best_carrier['delay_rate']*100:.1f}%")
    
    print(f"\n⚠️ WORST CARRIER (by cost per mile):")
    print(f"  • {worst_carrier['Carrier']}")
    print(f"  • Avg delay: {worst_carrier['avg_delay_min']:.1f} minutes")
    print(f"  • Cost per mile: ${worst_carrier['avg_cost_per_mile']:.2f}")
    print(f"  • Delay rate: {worst_carrier['delay_rate']*100:.1f}%")
    
    # Cost difference
    cost_diff = worst_carrier['avg_cost_per_mile'] - best_carrier['avg_cost_per_mile']
    pct_diff = (cost_diff / best_carrier['avg_cost_per_mile']) * 100
    
    print(f"\n💰 VALUE COMPARISON:")
    print(f"  • {worst_carrier['Carrier']} costs ${cost_diff:.2f} more per mile than {best_carrier['Carrier']}")
    print(f"  • That's {pct_diff:.0f}% higher cost per mile!")
    
    # Best and worst airports
    best_airport = airport_summary.iloc[0]
    worst_airport = airport_summary.iloc[-1]
    
    print(f"\n🏆 MOST EFFICIENT AIRPORT:")
    print(f"  • {best_airport['Airport']}")
    print(f"  • Avg delay: {best_airport['avg_delay_min']:.1f} minutes")
    print(f"  • Cost per mile: ${best_airport['avg_cost_per_mile']:.2f}")
    
    print(f"\n⚠️ LEAST EFFICIENT AIRPORT:")
    print(f"  • {worst_airport['Airport']}")
    print(f"  • Avg delay: {worst_airport['avg_delay_min']:.1f} minutes")
    print(f"  • Cost per mile: ${worst_airport['avg_cost_per_mile']:.2f}")
    
    print("\n" + "="*60)

## 10. Export Complete Dataset for Tableau

Export the full cleaned dataset with computed features for deeper Tableau analysis.

In [ ]:
if df is not None:
    # Select relevant columns for Tableau
    tableau_export = df[[
        'Year', 'Month', 'DayofMonth',
        'Carrier', 'Origin', 'Dest',
        'Distance', 'ArrDelay', 'DepDelay',
        'delay_cost', 'cost_per_mile', 'is_delayed'
    ]].copy()
    
    # Export
    tableau_export.to_csv('../outputs/full_dataset_for_tableau.csv', index=False)
    
    print(f"✓ Full dataset exported: {len(tableau_export):,} rows")
    print(f"✓ File: outputs/full_dataset_for_tableau.csv")
    print(f"✓ Size: {tableau_export.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

## 11. Next Steps for Tableau

**Files ready for Tableau:**
1. `outputs/airline_summary.csv` - Airline-level metrics
2. `outputs/airport_summary.csv` - Airport-level metrics
3. `outputs/full_dataset_for_tableau.csv` - Complete flight-level data

**Recommended Tableau Visualizations:**
1. **Bar Chart** - Average delay cost per airline (sorted)
2. **Heatmap** - Airport vs Carrier delay costs
3. **Line Chart** - Delay trends over time (if using multi-month data)
4. **Scatter Plot** - Cost per mile vs Delay rate
5. **KPI Cards** - Top performer, total cost impact, avg delay

**Interactive Filters:**
- Carrier selection
- Airport selection
- Month/time period
- Distance ranges